In [3]:
# 필요한 라이브러리 설치
!pip install xgboost

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 작업 디렉토리 설정 (본인의 데이터가 있는 경로로 수정해주세요)
import os
base_path = '/content/drive/My Drive/Colab Notebooks/homerun/'  # 이 경로를 본인의 데이터가 있는 경로로 수정해주세요

# 파일 경로 설정
arrival_data_path = os.path.join(base_path, 'arrival_data2.csv')
gstation_path = os.path.join(base_path, 'gStation2.csv')

# 경로 확인
print("현재 설정된 경로:")
print(f"arrival_data_path: {arrival_data_path}")
print(f"gstation_path: {gstation_path}")

# 파일 존재 여부 확인
print("\n파일 존재 여부 확인:")
print(f"arrival_data2.csv 존재 여부: {os.path.exists(arrival_data_path)}")
print(f"gStation2.csv 존재 여부: {os.path.exists(gstation_path)}")

Mounted at /content/drive
현재 설정된 경로:
arrival_data_path: /content/drive/My Drive/Colab Notebooks/homerun/arrival_data2.csv
gstation_path: /content/drive/My Drive/Colab Notebooks/homerun/gStation2.csv

파일 존재 여부 확인:
arrival_data2.csv 존재 여부: True
gStation2.csv 존재 여부: True


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# 시간 변환 함수
def time_to_minutes(time_str):
    """시간을 분으로 변환"""
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes

def minutes_to_time(minutes):
    """분을 시:분 형식으로 변환"""
    hours = minutes // 60
    mins = minutes % 60
    return f"{hours:02d}:{mins:02d}"

def get_time_period(minutes):
    """시간대 구분 (피크시간 등)"""
    hour = minutes // 60
    if 7 <= hour < 9:  # 아침 피크
        return 'morning_peak'
    elif 16 <= hour < 19:  # 저녁 피크
        return 'evening_peak'
    elif 12 <= hour < 14:  # 점심 시간
        return 'lunch_time'
    else:
        return 'normal_time'

def create_features(df):
    """피처 엔지니어링"""
    # 기본 시간 변환
    df['arrival_minutes'] = df['ARRIVAL_TIME'].apply(time_to_minutes)

    # 시간대 관련 피처
    df['hour'] = df['arrival_minutes'] // 60
    df['minute'] = df['arrival_minutes'] % 60
    df['time_period'] = df['arrival_minutes'].apply(get_time_period)

    # 15분 단위 구간
    df['quarter_hour'] = (df['minute'] // 15)

    # 피크시간 여부
    df['is_peak_hour'] = df['time_period'].isin(['morning_peak', 'evening_peak']).astype(int)

    # 요일 및 출발지 원핫 인코딩
    df = pd.get_dummies(df, columns=['DAY', 'DEPART_AT', 'time_period'])

    return df

# 데이터 로드 및 전처리
print("데이터 로드 중...")
arrival_df = pd.read_csv(arrival_data_path)
print("원본 데이터 형태:", arrival_df.shape)
print("\n원본 데이터 샘플:")
print(arrival_df.head())

# 피처 생성
print("\n피처 생성 중...")
processed_df = create_features(arrival_df)

# 필요없는 컬럼 제거
if 'ARRIVAL_TIME' in processed_df.columns:
    processed_df = processed_df.drop('ARRIVAL_TIME', axis=1)

print("\n전처리 후 데이터 형태:", processed_df.shape)
print("\n생성된 피처:")
print(processed_df.columns.tolist())

# 피처와 타겟 분리
X = processed_df.drop('arrival_minutes', axis=1)
y = processed_df['arrival_minutes']

# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("\n데이터 분할 완료:")
print("학습 데이터 형태:", X_train.shape)
print("테스트 데이터 형태:", X_test.shape)

데이터 로드 중...
원본 데이터 형태: (11, 3)

원본 데이터 샘플:
  ARRIVAL_TIME  DAY DEPART_AT
0         8:15  MON       STA
1         8:17  MON       STA
2        10:00  MON       SCH
3        10:05  MON       SCH
4        10:16  MON       STA

피처 생성 중...

전처리 후 데이터 형태: (11, 11)

생성된 피처:
['arrival_minutes', 'hour', 'minute', 'quarter_hour', 'is_peak_hour', 'DAY_MON', 'DEPART_AT_SCH', 'DEPART_AT_STA', 'time_period_evening_peak', 'time_period_morning_peak', 'time_period_normal_time']

데이터 분할 완료:
학습 데이터 형태: (8, 10)
테스트 데이터 형태: (3, 10)


In [7]:
# XGBoost 모델 정의
model = XGBRegressor(
    n_estimators=200,
    max_depth=7,
    learning_rate=0.01,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='mae'
)

# 모델 학습
print("모델 학습 중...")
eval_set = [(X_test, y_test)]
model.fit(
    X_train,
    y_train,
    verbose=False
)

# 예측
y_pred = model.predict(X_test)

# 평가 메트릭스
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n모델 성능 평가:")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Mean Absolute Error: {mae:.2f} minutes")
print(f"Root Mean Squared Error: {np.sqrt(mse):.2f} minutes")
print(f"R² Score: {r2:.4f}")

# 피처 중요도 확인
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n상위 10개 중요 피처:")
print(feature_importance.head(10))

# 예측 값을 시간으로 변환
def round_minutes(minutes):
    """예측된 분을 5분 단위로 반올림"""
    return round(minutes / 5) * 5

# 예측 결과 샘플 확인 (시간 형식으로 변환하여 표시)
comparison_df = pd.DataFrame({
    'Actual_Minutes': y_test,
    'Predicted_Minutes': [round_minutes(x) for x in y_pred],  # 5분 단위로 반올림
    'Difference_Minutes': y_test - y_pred
})

comparison_df['Actual_Time'] = comparison_df['Actual_Minutes'].apply(minutes_to_time)
comparison_df['Predicted_Time'] = comparison_df['Predicted_Minutes'].apply(minutes_to_time)

print("\n예측 결과 샘플 (처음 5개):")
print(comparison_df.head())

# 예측 정확도 분석
print("\n예측 정확도 분석:")
print(f"5분 이내 정확도: {(abs(comparison_df['Difference_Minutes']) <= 5).mean()*100:.2f}%")
print(f"10분 이내 정확도: {(abs(comparison_df['Difference_Minutes']) <= 10).mean()*100:.2f}%")
print(f"15분 이내 정확도: {(abs(comparison_df['Difference_Minutes']) <= 15).mean()*100:.2f}%")

모델 학습 중...

모델 성능 평가:
Mean Squared Error: 10156.95
Mean Absolute Error: 90.92 minutes
Root Mean Squared Error: 100.78 minutes
R² Score: 0.8350

상위 10개 중요 피처:
                    feature  importance
0                      hour    0.406788
7  time_period_evening_peak    0.406710
3              is_peak_hour    0.145298
1                    minute    0.041205
2              quarter_hour    0.000000
4                   DAY_MON    0.000000
5             DEPART_AT_SCH    0.000000
6             DEPART_AT_STA    0.000000
8  time_period_morning_peak    0.000000
9   time_period_normal_time    0.000000

예측 결과 샘플 (처음 5개):
   Actual_Minutes  Predicted_Minutes  Difference_Minutes Actual_Time  \
5            1010                955           56.434204       16:50   
0             495                645         -152.260437       08:15   
9            1032                970           64.053101       17:12   

  Predicted_Time  
5          15:55  
0          10:45  
9          16:10  

예측 정확도 분석:
5분 이내 

In [8]:
# gStation 데이터 로드
print("기존 스케줄 데이터 로드 중...")
gstation_df = pd.read_csv(gstation_path)
new_schedule = gstation_df.copy()
days = ['MON', 'TUE', 'WED', 'THU', 'FRI']

# 각 요일, 시간별로 새로운 도착 시간 예측
for day in days:
    print(f"\n{day} 처리 중...")
    school_times = gstation_df[f'{day}_SCHOOL_DEPART']

    for idx, time in school_times.items():
        # 기본 피처 생성
        depart_minutes = time_to_minutes(time)
        hour = depart_minutes // 60
        minute = depart_minutes % 60
        time_period = get_time_period(depart_minutes)

        # 예측을 위한 데이터프레임 생성
        pred_data = pd.DataFrame({
            'hour': [hour],
            'minute': [minute],
            'quarter_hour': [minute // 15],
            'is_peak_hour': [1 if time_period in ['morning_peak', 'evening_peak'] else 0]
        })

        # 요일 더미 변수 추가
        for d in days:
            pred_data[f'DAY_{d}'] = 1 if d == day else 0

        # 출발지 더미 변수 추가
        pred_data['DEPART_AT_SCH'] = 1
        pred_data['DEPART_AT_STA'] = 0

        # 시간대 더미 변수 추가
        for period in ['morning_peak', 'evening_peak', 'lunch_time', 'normal_time']:
            pred_data[f'time_period_{period}'] = 1 if period == time_period else 0

        # 컬럼 순서 맞추기
        pred_data = pred_data.reindex(columns=X_train.columns, fill_value=0)

        # 예측
        predicted_minutes = int(model.predict(pred_data)[0])
        predicted_time = minutes_to_time(predicted_minutes)

        # 스케줄 업데이트
        new_schedule.at[idx, f'{day}_STATION_DEPART'] = predicted_time

# 결과 저장
output_path = os.path.join(base_path, 'new_schedule_improved.csv')
new_schedule.to_csv(output_path, index=False)
print(f"\n새로운 스케줄이 저장되었습니다: {output_path}")

# 결과 비교 분석
print("\n새로운 스케줄 샘플 (처음 5행):")
print(new_schedule.head())

# 원본과 새로운 스케줄 비교 (첫번째 요일)
print("\n원본과 새로운 스케줄 비교 (MON):")
comparison = pd.DataFrame({
    '원본_출발': gstation_df['MON_STATION_DEPART'],
    '새로운_출발': new_schedule['MON_STATION_DEPART'],
    '학교출발시각': gstation_df['MON_SCHOOL_DEPART']
})
print(comparison.head(10))

# 변경된 시간 분석
def time_diff_minutes(time1, time2):
    return time_to_minutes(time1) - time_to_minutes(time2)

comparison['시간차(분)'] = comparison.apply(
    lambda row: time_diff_minutes(row['새로운_출발'], row['원본_출발']), axis=1)

print("\n시간 변경 통계:")
print(f"평균 변경 시간: {comparison['시간차(분)'].mean():.2f}분")
print(f"최대 변경 시간: {comparison['시간차(분)'].max():.2f}분")
print(f"최소 변경 시간: {comparison['시간차(분)'].min():.2f}분")

기존 스케줄 데이터 로드 중...

MON 처리 중...

TUE 처리 중...

WED 처리 중...

THU 처리 중...

FRI 처리 중...

새로운 스케줄이 저장되었습니다: /content/drive/My Drive/Colab Notebooks/homerun/new_schedule_improved.csv

새로운 스케줄 샘플 (처음 5행):
  MON_SCHOOL_DEPART MON_STATION_DEPART TUE_SCHOOL_DEPART TUE_STATION_DEPART  \
0              7:45              10:49              7:45              10:49   
1              8:00              10:47              8:00              10:47   
2              8:02              10:47              8:02              10:47   
3              8:03              10:47              8:03              10:47   
4              8:05              10:47              8:05              10:47   

  WED_SCHOOL_DEPART WED_STATION_DEPART THU_SCHOOL_DEPART THU_STATION_DEPART  \
0              7:45              10:49              7:45              10:49   
1              8:00              10:47              8:00              10:47   
2              8:02              10:47              8:02              10:47   
3          